# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-02 01:03:06] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-02 01:03:06] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-02 01:03:06] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-02 01:03:08] INFO server_args.py:1599: Attention backend not specified. Use fa3 backend by default.


[2026-01-02 01:03:08] INFO server_args.py:2474: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=64 avail_mem=58.77 GB):  40%|████      | 8/20 [00:00<00:00, 14.67it/s]

Capturing batches (bs=56 avail_mem=58.77 GB):  40%|████      | 8/20 [00:00<00:00, 14.67it/s]

Capturing batches (bs=32 avail_mem=58.73 GB):  60%|██████    | 12/20 [00:01<00:00,  9.03it/s]

Capturing batches (bs=16 avail_mem=58.72 GB):  70%|███████   | 14/20 [00:01<00:00,  9.40it/s]

Capturing batches (bs=1 avail_mem=76.72 GB): 100%|██████████| 20/20 [00:01<00:00, 11.06it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Anna and I am an engineer in a company called Sharp Instruments Inc. I have always had a passion for helping people. My degrees from the University of Virginia and Georgia Institute of Technology are in electrical engineering. I have been working with computers for the past 15 years, which is where my expertise comes from. I have been involved in a variety of projects such as computer programming, software development, and web design. At Sharp Instruments, I am responsible for the software and hardware development for our product line. My work includes design, prototyping, testing, and deployment of the products to customers. I have also designed and implemented various software
Prompt: The president of the United States is
Generated text:  a male. A certain organization president is a female. A black person is a male. Therefore, a male black person is a female.

Is the argument valid?
To determine whether the argument is valid, we need to ana

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Occupation]. I'm a [Type of Vehicle] with [Number of Wheels] wheels. I'm [Favorite Food] and I enjoy [Favorite Activity]. I'm [Favorite Book] and I love [Favorite Movie]. I'm [Favorite Music] and I listen to [Favorite Album] every day. I'm [Favorite Sport] and I play [Favorite Game]. I'm [Favorite Place] and I love [Favorite Thing]. I'm [Favorite Animal] and I have [Number of Pets] pets. I'm [Favorite Color] and I wear [Number

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. It is also home to the French Parliament and the French National Museum of Modern Art. Paris is a bustling city with a rich history and culture, and is a popular tourist destination. Its name comes from the Latin word for "city" and is often used as a reference to the city's importance and influence. Paris is also known for its fashion industry, with many famous designers and boutiques located in the city. The city is home to many cultural institutions, including the Louvre Museum, the Musée d'

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in several key areas, including:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn and adapt to human behavior and preferences. This could lead to more natural and intuitive interactions between humans and machines.

2. Enhanced machine learning capabilities: AI is likely to become even more powerful and capable, with the ability to learn from vast amounts of data and make more accurate predictions and decisions. This could lead to more efficient and effective decision-making in a wide range of applications.

3. Increased focus on ethical considerations: As AI becomes more integrated with human intelligence



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Your Name], and I'm [Your Age] years old. I'm a [Your Occupation] who's always been curious about the world around me. I'm always trying to learn new things and take in the sights and sounds of the place I'm based in. I love to explore, discover, and stay curious. How are you today? I hope you enjoy your day! I'm [Your Name], a [Your Occupation] who is always looking for new things to learn and experiences to try. My name is [Your Name], and I'm always on the lookout for a challenge that I can try out. What are

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. 

Here's a concise factual statement about France's capital city, using only the words and phrases provided:

"Paris, the capital, is a city in France." 

Here's a more detailed statement:

The capital city of France is Paris. 

Here's an addition

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

 This

 could

 include

 self

-driving

In [6]:
llm.shutdown()